# fasterrcnn_resnet50_fpn

In [ ]:
import torch
import torchvision
import cv2
import numpy

https://pytorch.org/vision/main/models/generated/torchvision.models.detection.fasterrcnn_resnet50_fpn.html

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
threshold = 0.99

In [ ]:
def people_on(image):
    global model
    model = model.eval()
    image = torch\
        .from_numpy(image.copy().astype('float32'))\
        .permute(2, 0, 1) / 255.0
    predictions = model(image[None, ...])
    boxes = predictions[0]['boxes'][
        (predictions[0]['scores'] > threshold)
        & (predictions[0]['labels'] == 1)
    ]
    return boxes.detach().numpy()

In [ ]:
def draw_boxes(boxes, image):
    image = numpy.array(image)
    for x, y, w, h in boxes:
        x, y, w, h = map(int, (x, y, w, h))
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
    cv2.putText(image, f'Count: {len(boxes)}', (40, 70), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 2)
    return image

In [ ]:
def detect(image, filters=[]):
    for filter in filters:
        image = filter(image)
    return draw_boxes(people_on(image), image)

In [ ]:
def image_from_camera():
    video = cv2.VideoCapture(0)
    _, image = video.read()
    video.release()
    return image

In [ ]:
def image_from_file(filepath):
    return cv2.imread(filepath)

In [ ]:
from IPython.display import display, Image

In [ ]:
def show(image):
    window = display(None, display_id=True)
    _, image = cv2.imencode('.jpeg', image)
    window.update(Image(data = image.tobytes()))

In [ ]:
def grayscale(image):
    return cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2RGB)

In [ ]:
show(detect(image_from_file('res/easy-people-5.jpg'), []))

In [ ]:
show(detect(image_from_file('res/hard-people-7.jpg'), []))

In [ ]:
show(detect(image_from_file('res/medium-people-6.jpg'), []))

In [ ]:
show(detect(image_from_file('res/medium-people-5-dark.jpg'), []))

In [ ]:
show(detect(image_from_file('res/hard-people-many.jpg'), []))

In [ ]:
show(detect(image_from_camera()))